In [ ]:
import geemap
import ee
Map = geemap.Map()
from time import sleep
import csv
def deleteContent(fName):
    with open(fName, "w"):
        pass
deleteContent('竞赛.csv')
f = open('竞赛.csv','w',encoding='utf-8')
csv_writer = csv.writer(f)
Map

In [ ]:
'''Adding vector data'''
duchang_shp = r"G:\competition\水稻有关数据\都昌矢量数据\都昌矢量数据.shp"
duchang = geemap.shp_to_ee(duchang_shp)
Map.centerObject(duchang, zoom=10)
roi = duchang.geometry()  
'''Date'''
Double_1_1 = ['04-01','05-10']
Double_1_2 = ['05-11','06-10']
Double_1_3 = ['06-11','07-10']
Double_2_1 = ['07-11','08-20']
Double_2_2 = ['08-21','09-20']
Double_2_3 = ['09-21','10-20']
Names = ['移栽分节期1','拔节孕穗期1','抽穗成熟期1','移栽分节期2','拔节孕穗期2','抽穗成熟期2']
# Single_1_1 = ['06-21','07-30']
# Single_1_2 = ['08-01','08-30']
# Single_1_3 = ['09-01','09-30']
D_list = [Double_1_1,Double_1_2,Double_1_3,Double_2_1,Double_2_2,Double_2_3]

palette2 = ['FFFFFF,CE7E45,DF923D,F1B555,FCD163,99B718,74A901']
ndvi_visparam = {'min':-1,'max':1,'palette':['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']}
L5_visualization = {'bands': ['SR_B3', 'SR_B2', 'SR_B1'],'min': 0.0,'max': 0.3}
L7_visualization = {'bands': ['SR_B3', 'SR_B2', 'SR_B1'],'min': 0.0,'max': 0.3}
L8_visualization = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'min': 0.0,'max': 0.3}
elev_args = {'crs': 'EPSG:3857','dimensions': '300','region': roi,'min': -2000,'max': 10000,'palette': 'green, blanchedalmond,orange,black ','framesPerSecond': 12}
classVis = {'min': 0,'max': 10,'palette': ['006400' ,'ffbb22', 'ffff4c', 'f096ff', 'fa0000', 'b4b4b4','f0f0f0', '0064c8', '0096a0', '00cf75', 'fae6a0']}

In [ ]:

'''Add land use data-remap-sample-trainingSample-validationSample '''
LUCC = ee.ImageCollection("ESA/WorldCover/v100")
# print(LUCC_10.size().getInfo())  #1
LUCC = LUCC.filterBounds(roi).first()
#Remap the land cover class values to a 0-based sequential series.
classValues = [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100]
remapValues = ee.List.sequence(0, 10)
label = 'lc'
LUCC = LUCC.remap(classValues, remapValues).rename(label).toByte() #Casts the input value to an unsigned 8-bit integer.
Landsat8_col = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")  #2013-2021
Landsat7_col = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")  #2000-2021
Landsat5_col = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")  #1984-2011
Sentinel_col  = ee.ImageCollection("COPERNICUS/S2_SR")  #2017-2021
SRTM30 = ee.Image("USGS/SRTMGL1_003").select('elevation')
slope = ee.Terrain.slope(SRTM30).clip(roi).rename('slope')
'''Add layer'''
Map.addLayer(SRTM30,elev_args,'SRTM30')
Map.addLayer(LUCC, classVis, "LUCC")
Map.addLayer(roi, {'color': 'white'}, 'ROI', False, 0.5)

In [ ]:
def clipp(image):
     return image.clip(roi)
    
def L8_applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True) \
              .addBands(thermalBands, None, True)    
def Sn_applyScaleFactors(image):
    opticalBands = image.select('B.*').multiply(0.0001)
    thermalBands = image.select(['AOT','WVP']).multiply(0.001)
    return image.addBands(opticalBands, None, True) \
              .addBands(thermalBands, None, True)    
def L7_applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True) \
              .addBands(thermalBand, None, True)
def L5_applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B6').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True) \
              .addBands(thermalBands, None, True)    

def L8_maskL8sr(image):
      #<<表示二进制左移几位
      # 二进制第三位表示有云阴影，第五位表示有云。
    cloudShadowBitMask = 1 << 4
    cloudsBitMask = 1 << 3
    #选择质量评估波段
    #按位与运算，将检测出云以及云阴影像元值为0
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)\
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    #掩膜云以及云阴影，并选择波段1-9将单位变为0-1之间，并复制影像时间属性。
    return image.updateMask(mask) \
      .select("SR_B[0-9]*") \
      .copyProperties(image, ["system:time_start"])
def Sn_maskL8sr(image):
    qa = image.select('QA60')
# Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
# Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).copyProperties(image, ["system:time_start"])
def L7_maskL8sr(image):
      #<<表示二进制左移几位
      # 二进制第三位表示有云阴影，第五位表示有云。
    cloudShadowBitMask = 1 << 4
    cloudsBitMask = 1 << 3
    #选择质量评估波段
    qa = image.select('QA_PIXEL')
    #按位与运算，将检测出云以及云阴影像元值为0
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)\
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    #掩膜云以及云阴影，并选择波段1-9将单位变为0-1之间，并复制影像时间属性。
    return image.updateMask(mask) \
      .select("SR_B[0-9]*") \
      .copyProperties(image, ["system:time_start"])                      
def L5_maskL8sr(image):
      #<<表示二进制左移几位
      # 二进制第三位表示有云阴影，第五位表示有云。
    cloudShadowBitMask = 1 << 4
    cloudsBitMask = 1 << 3
    #选择质量评估波段
    qa = image.select('QA_PIXEL')
    #按位与运算，将检测出云以及云阴影像元值为0
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)\
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    #掩膜云以及云阴影，并选择波段1-9将单位变为0-1之间，并复制影像时间属性。
    return image.updateMask(mask) \
      .select("SR_B[0-9]*") \
      .copyProperties(image, ["system:time_start"]) 

def Cal_L8_NDVI(image):
    ndvi = image.normalizedDifference(["SR_B5","SR_B4"]).rename('RNDVI')
    return ndvi.copyProperties(image, ["system:time_start"])
def Cal_Sn_NDVI(image):
    ndvi = image.normalizedDifference(["B8","B4"]).rename('RNDVI')
    return ndvi.copyProperties(image, ["system:time_start"])
def Cal_L7_NDVI(image):
    ndvi = image.normalizedDifference(["SR_B4","SR_B3"]).rename('RNDVI')
    return ndvi.copyProperties(image, ["system:time_start"])
def Cal_L5_NDVI(image):
    ndvi = image.normalizedDifference(["SR_B4","SR_B3"]).rename('RNDVI')
    return ndvi.copyProperties(image, ["system:time_start"])

def Cal_L8_EVI(image):
    evi = image.expression(
    '2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1)',
    {
        'red': image.select('SR_B4'),    # 620-670nm, RED
        'nir': image.select('SR_B5'),    # 841-876nm, NIR
        'blue': image.select('SR_B2')    # 459-479nm, BLUE
    }).rename('REVI')
    return evi.copyProperties(image, ["system:time_start"])
def Cal_Sn_EVI(image):
    evi = image.expression(
    '2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1)',
    {
        'red': image.select('B4'),    # 620-670nm, RED
        'nir': image.select('B8'),    # 841-876nm, NIR
        'blue': image.select('B2')    # 459-479nm, BLUE
    }).rename('REVI')
    return evi.copyProperties(image, ["system:time_start"])
def Cal_L7_EVI(image):
    evi = image.expression(
    '2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1)',
    {
        'red': image.select('SR_B3'),    # 620-670nm, RED
        'nir': image.select('SR_B4'),    # 841-876nm, NIR
        'blue': image.select('SR_B1')    # 459-479nm, BLUE
    }).rename('REVI')
    return evi.copyProperties(image, ["system:time_start"])
def Cal_L5_EVI(image):
    evi = image.expression(
    '2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1)',
    {
        'red': image.select('SR_B3'),    # 620-670nm, RED
        'nir': image.select('SR_B4'),    # 841-876nm, NIR
        'blue': image.select('SR_B1')    # 459-479nm, BLUE
    }).rename('REVI')
    return evi.copyProperties(image, ["system:time_start"])

def Cal_L8_LSWI(image):
    lswi = image.normalizedDifference(["SR_B5","SR_B6"]).rename('RLSWI')
    return lswi.copyProperties(image, ["system:time_start"])
def Cal_Sn_LSWI(image):
    lswi = image.normalizedDifference(["B8","B11"]).rename('RLSWI')
    return lswi.copyProperties(image, ["system:time_start"])
def Cal_L7_LSWI(image):
    lswi = image.normalizedDifference(["SR_B4","SR_B5"]).rename('RLSWI')
    return lswi.copyProperties(image, ["system:time_start"])
def Cal_L5_LSWI(image):
    lswi = image.normalizedDifference(["SR_B4","SR_B5"]).rename('RLSWI')
    return lswi.copyProperties(image, ["system:time_start"])

def Cal_L8_NDWI(image):
    ndwi = image.normalizedDifference(['SR_B3','SR_B5']).rename('RNDWI')
    return ndwi.copyProperties(image, ["system:time_start"])
def Cal_Sn_NDWI(image):
    ndwi = image.normalizedDifference(['B3','B8']).rename('RNDWI')
    return ndwi.copyProperties(image, ["system:time_start"])
def Cal_L7_NDWI(image):
    ndwi = image.normalizedDifference(['SR_B2','SR_B4']).rename('RNDWI')
    return ndwi.copyProperties(image, ["system:time_start"])
def Cal_L5_NDWI(image):
    ndwi = image.normalizedDifference(['SR_B2','SR_B4']).rename('RNDWI')
    return ndwi.copyProperties(image, ["system:time_start"])

def Cal_L8_GCVI(image):
    gcvi = image.expression(
    'nir / green - 1',
    {
        'nir': image.select('SR_B5'),    
        'green': image.select('SR_B3'),    
    }).rename('RGCVI')
    return gcvi.copyProperties(image, ["system:time_start"])
def Cal_Sn_GCVI(image):
    gcvi = image.expression(
    'nir / green - 1',
    {
        'nir': image.select('B8'),    
        'green': image.select('B3'),    
    }).rename('RGCVI')
    return gcvi.copyProperties(image, ["system:time_start"])
def Cal_L7_GCVI(image):
    gcvi = image.expression(
    'nir / green - 1',
    {
        'nir': image.select('SR_B4'),    
        'green': image.select('SR_B2'),    
    }).rename('RGCVI')
    return gcvi.copyProperties(image, ["system:time_start"])
def Cal_L5_GCVI(image):
    gcvi = image.expression(
    'nir / green - 1',
    {
        'nir': image.select('SR_B4'),    
        'green': image.select('SR_B2'),    
    }).rename('RGCVI')
    return gcvi.copyProperties(image, ["system:time_start"])

def Cal_L8_RVI(image):
    rvi = image.expression(
    'nir / red',
    {
        'nir': image.select('SR_B5'),    
        'red': image.select('SR_B4'),    
    }).rename('RRVI')
    return rvi.copyProperties(image, ["system:time_start"])
def Cal_Sn_RVI(image):
    rvi = image.expression(
    'nir / red',
    {
        'nir': image.select('B8'),    
        'red': image.select('B4'),    
    }).rename('RRVI')
    return rvi.copyProperties(image, ["system:time_start"])
def Cal_L7_RVI(image):
    rvi = image.expression(
    'nir / red',
    {
        'nir': image.select('SR_B4'),    
        'red': image.select('SR_B3'),    
    }).rename('RRVI')
    return rvi.copyProperties(image, ["system:time_start"])
def Cal_L5_RVI(image):
    rvi = image.expression(
    'nir / red',
    {
        'nir': image.select('SR_B4'),    
        'red': image.select('SR_B3'),    
    }).rename('RRVI')
    return rvi.copyProperties(image, ["system:time_start"])

def Cal_L8_EVI2(image):
    evi2 = image.expression(
    '2.5 * (nir - red) / (nir + 2.4 * red  + 1)',
    {
        'red': image.select('SR_B4'),    # 620-670nm, RED
        'nir': image.select('SR_B5'),    # 841-876nm, NIR
    }).rename('REVI2')
    return evi2.copyProperties(image, ["system:time_start"])
def Cal_Sn_EVI2(image):
    evi2 = image.expression(
    '2.5 * (nir - red) / (nir + 2.4 * red  + 1)',
    {
        'red': image.select('B4'),    # 620-670nm, RED
        'nir': image.select('B8'),    # 841-876nm, NIR
    }).rename('REVI2')
    return evi2.copyProperties(image, ["system:time_start"])
def Cal_L7_EVI2(image):
    evi2 = image.expression(
    '2.5 * (nir - red) / (nir + 2.4 * red  + 1)',
    {
        'red': image.select('SR_B3'),    # 620-670nm, RED
        'nir': image.select('SR_B4'),    # 841-876nm, NIR
    }).rename('REVI2')
    return evi2.copyProperties(image, ["system:time_start"])
def Cal_L5_EVI2(image):
    evi2 = image.expression(
    '2.5 * (nir - red) / (nir + 2.4 * red  + 1)',
    {
        'red': image.select('SR_B3'),    # 620-670nm, RED
        'nir': image.select('SR_B4'),    # 841-876nm, NIR
    }).rename('REVI2')
    return evi2.copyProperties(image, ["system:time_start"])

def Cal_L8_SAVI(image):
    savi = image.expression(
    '1.5 * (nir - red) / (nir +  red  + 0.5)',
    {
        'red': image.select('SR_B4'),    # 620-670nm, RED
        'nir': image.select('SR_B5'),    # 841-876nm, NIR
    }).rename('RSAVI')
    return savi.copyProperties(image, ["system:time_start"])
def Cal_Sn_SAVI(image):
    savi = image.expression(
    '1.5 * (nir - red) / (nir +  red  + 0.5)',
    {
        'red': image.select('B4'),    # 620-670nm, RED
        'nir': image.select('B8'),    # 841-876nm, NIR
    }).rename('RSAVI')
    return savi.copyProperties(image, ["system:time_start"])
def Cal_L7_SAVI(image):
    savi = image.expression(
    '1.5 * (nir - red) / (nir +  red  + 0.5)',
    {
        'red': image.select('SR_B3'),    # 620-670nm, RED
        'nir': image.select('SR_B4'),    # 841-876nm, NIR
    }).rename('RSAVI')
    return savi.copyProperties(image, ["system:time_start"])
def Cal_L5_SAVI(image):
    savi = image.expression(
    '1.5 * (nir - red) / (nir +  red  + 0.5)',
    {
        'red': image.select('SR_B3'),    # 620-670nm, RED
        'nir': image.select('SR_B4'),    # 841-876nm, NIR
    }).rename('RSAVI')
    return savi.copyProperties(image, ["system:time_start"])

I_list = ['RNDVI','REVI','RLSWI','RNDWI','RGCVI','RRVI','RSAVI']
Cal_L8_list = [Cal_L8_NDVI,Cal_L8_EVI,Cal_L8_LSWI,Cal_L8_NDWI,Cal_L8_GCVI,Cal_L8_RVI,Cal_L8_SAVI]
L8_Bands_list = I_list
Cal_L7_list = [Cal_L7_NDVI,Cal_L7_EVI,Cal_L7_LSWI,Cal_L7_NDWI,Cal_L7_GCVI,Cal_L7_RVI,Cal_L7_SAVI]
L7_Bands_list = I_list
Cal_L5_list = [Cal_L5_NDVI,Cal_L5_EVI,Cal_L5_LSWI,Cal_L5_NDWI,Cal_L5_GCVI,Cal_L5_RVI,Cal_L5_SAVI]
L5_Bands_list = I_list
Cal_Sn_list = [Cal_Sn_NDVI,Cal_Sn_EVI,Cal_Sn_LSWI,Cal_Sn_NDWI,Cal_Sn_GCVI,Cal_Sn_RVI,Cal_Sn_SAVI]
Sn_Bands_list = I_list

In [ ]:
'''Main'''
sy = 2017
ey = 2021

'''Processing'''
for Da,name in (zip(D_list,Names):  #循环每个时期
    sleep(0.01)
    all_cols = ee.Image()  #将所有指数最后放在一个影像中
    for Cal_L8,L8_Band,Cal_L7,L7_Band,Cal_L5,L5_Band,Cal_Sn,Sn_Band in zip(Cal_L8_list,L8_Bands_list,Cal_L7_list,L7_Bands_list,Cal_L5_list,L5_Bands_list,Cal_Sn_list,Sn_Bands_list):   #循环每个指数
        cols_list = ee.List([])  #将每年同一时期的影像放在一起
        for year in range(sy, ey+1, 1):
            L_8 = Landsat8_col.filterDate(str(year) + '-' + Da[0],str(year) + '-' + Da[1])\
            .filterBounds(roi).map(L8_applyScaleFactors).select("SR_B.*|QA_P.*").map(clipp).map(L8_maskL8sr)

            L_7 = Landsat7_col.filterDate(str(year) + '-' + Da[0],str(year) + '-' + Da[1])\
            .filterBounds(roi).map(L7_applyScaleFactors).select("SR_B.*|QA_P.*").map(clipp).map(L7_maskL8sr)

            L_5 = Landsat5_col.filterDate(str(year) + '-' + Da[0],str(year) + '-' + Da[1])\
            .filterBounds(roi).map(L5_applyScaleFactors).select("SR_B.*|QA_P.*").map(clipp).map(L5_maskL8sr)

            Sen = Sentinel_col.filterDate(str(year) + '-' + Da[0],str(year) + '-' + Da[1])\
            .filterBounds(roi).map(Sn_applyScaleFactors).select("B.*|QA60").map(clipp).map(Sn_maskL8sr)

            L_8_I = L_8.map(Cal_L8).qualityMosaic(L8_Band)   #计算每个指数
            L_7_I = L_7.map(Cal_L7).qualityMosaic(L7_Band)   #计算每个指数
            L_5_I = L_5.map(Cal_L5).qualityMosaic(L5_Band)   #计算每个指数
            Sn_I = Sen.map(Cal_Sn).qualityMosaic(Sn_Band)   #计算每个指数
            cols_list = cols_list.add(L_8_I).add(L_7_I).add(Sn_I).add(L_5_I)  #添加
        cols = ee.ImageCollection.fromImages(cols_list)   #将列表转为影像集
#         print(' List of  Date: ',cols.aggregate_array("system:time_start").getInfo())
        col = cols.mosaic()  #多年同一时期融合成有值的一张影像
        all_cols = all_cols.addBands(col)  #添加
    all_cols = all_cols.select("R.*")  #选择目的波段
    all_cols = all_cols.addBands(SRTM30).addBands(slope)   #添加dem和slope辅助数据
    #Add land cover as a band of the reflectance image and sample 100 pixels at
    #5 m scale from each land cover class within a region of interest.
    sample = all_cols.addBands(LUCC).stratifiedSample(numPoints=100,classBand=label,region= roi,scale= 5,geometries= True)  #创建采样点
#     print(sample.getInfo())
    #Add a random value field to the sample and use it to approximately split 80%
    #of the features into a training set and 20% into a validation set.
    sample = sample.randomColumn()   #随机排列
    trainingSample = sample.filter('random <= 0.8')  #选取前80%个样点作为训练
    validationSample = sample.filter('random > 0.8')  #选取后20%个样点作为验证
#         print(' Bandnames:  ' + name + ': ',all_cols.bandNames().getInfo())
    # Train a 10-tree random forest classifier from the training sample.
    trainedClassifier = ee.Classifier.smileRandomForest(10).train(features = trainingSample,classProperty = label,inputProperties = all_cols.bandNames())
    #Get information about the trained classifier.
#         csv_writer.writerow(['Results of trained classifier '+ name + ': ', trainedClassifier.explain()])
    #Get a confusion matrix and overall accuracy for the training sample.
    trainAccuracy = trainedClassifier.confusionMatrix()
    csv_writer.writerow(['Training error matrix ' + name + ': ',str(trainAccuracy.getInfo())])
    csv_writer.writerow(['Training overall accuracy '+ name + ': ', str(trainAccuracy.accuracy().getInfo())])
    csv_writer.writerow(['Training kappa accuracy ' + name + ': ', str(trainAccuracy.kappa().getInfo())])
    #Get a confusion matrix and overall accuracy for the validation sample.
    validationSample = validationSample.classify(trainedClassifier)
    validationAccuracy = validationSample.errorMatrix(label, 'classification')
    csv_writer.writerow(['Validation error matrix '+ name + ': ', str(validationAccuracy.getInfo())])
    csv_writer.writerow(['Validation overall accuracy '+ name + ': ', str(validationAccuracy.accuracy().getInfo())])
    csv_writer.writerow(['Validation kappa accuracy ' + name + ': ', str(validationAccuracy.kappa().getInfo())])
    imgClassified = all_cols.classify(trainedClassifier)
    csv_writer.writerow([])
    Export.image.toDrive(image = imgClassified,description = name + '_Classifyed',crs =  "EPSG:4326",scale =  30,region =  roi,maxPixels =  1e13,folder =  '竞赛')
f.close()